# 

In [1]:
from captum.attr import LayerIntegratedGradients

In [2]:
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig

In [3]:
# there are some warning from transformer
# due to its verbose, disable

from transformers import logging
logging.set_verbosity(40)

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [6]:
import pandas as pd
from tqdm.autonotebook import tqdm

In [7]:
from linevul_model import Model
from linevul_helpers import TextDataset

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [9]:
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
config.num_attention_heads = 12

In [10]:
# get from LineVul
checkpoint = '/home/hqn650/LineVul/linevul/saved_models/checkpoint-best-f1/12heads_linevul_model.bin'

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

In [12]:
pre_train = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', 
                                                             config=config, 
                                                             ignore_mismatched_sizes=True)

In [13]:
from dataclasses import dataclass

@dataclass
class Args:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_gpu = torch.cuda.device_count()
    use_non_pretrained_model = False
    block_size = 512
    test_data_file = '/home/hqn650/LineVul/data/big-vul_dataset/test.csv'
    code_length=256
    do_local_explanation=True
    reasoning_method='attention'
    seed=42
    num_attention_heads=12
    do_sorting_by_line_scores=False
    do_sorting_by_pred_prob=False
    top_k_constant=10
    use_word_level_tokenizer=False
    eval_batch_size=512
    
args = Args()

In [14]:
model = Model(pre_train, config, tokenizer, args)

In [15]:
model.load_state_dict(torch.load(checkpoint, map_location=args.device))
model.to(args.device)

Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [16]:
test_dataset = TextDataset(tokenizer, args, file_type='test')

  0%|          | 0/18864 [00:00<?, ?it/s]

In [17]:
best_threshold=0.5

In [18]:
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=args.eval_batch_size, num_workers=0)

In [19]:
# multi-gpu evaluate
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [22]:
def linevul_predict(model, dataloader):
    model.eval()
    logits=[]  
    y_trues=[]
    for batch in test_dataloader:
        (inputs_ids, labels) = [x.to(args.device) for x in batch]
        with torch.no_grad():
            lm_loss, logit = model(input_ids=inputs_ids, labels=labels)
            logits.append(logit.cpu().numpy())
            y_trues.append(labels.cpu().numpy())
    # calculate scores
    logits = np.concatenate(logits, 0)
    y_trues = np.concatenate(y_trues, 0)
    y_preds = logits[:, 1] > best_threshold
    acc = accuracy_score(y_trues, y_preds)
    recall = recall_score(y_trues, y_preds)
    precision = precision_score(y_trues, y_preds)   
    f1 = f1_score(y_trues, y_preds)             
    result = {
        "test_accuracy": float(acc),
        "test_recall": float(recall),
        "test_precision": float(precision),
        "test_f1": float(f1),
        "test_threshold":best_threshold,
    }
    return result, y_trues, y_preds

In [23]:
result, y_trues, y_preds = linevul_predict(model, test_dataloader)

In [ ]:
# nb_eval_steps = 0
# model.eval()
# logits=[]  
# y_trues=[]
# for batch in test_dataloader:
#     (inputs_ids, labels) = [x.to(args.device) for x in batch]
#     with torch.no_grad():
#         lm_loss, logit = model(input_ids=inputs_ids, labels=labels)
#         logits.append(logit.cpu().numpy())
#         y_trues.append(labels.cpu().numpy())
#     nb_eval_steps += 1
# # calculate scores
# logits = np.concatenate(logits, 0)
# y_trues = np.concatenate(y_trues, 0)
# y_preds = logits[:, 1] > best_threshold
# acc = accuracy_score(y_trues, y_preds)
# recall = recall_score(y_trues, y_preds)
# precision = precision_score(y_trues, y_preds)   
# f1 = f1_score(y_trues, y_preds)             
# result = {
#     "test_accuracy": float(acc),
#     "test_recall": float(recall),
#     "test_precision": float(precision),
#     "test_f1": float(f1),
#     "test_threshold":best_threshold,
# }

In [ ]:
result

In [ ]:
correct_indices = np.where((y_trues == y_preds))
correct_indices = list(correct_indices[0])

In [ ]:
tp_indices = np.where((y_trues == y_preds) & (y_trues == 1))
tp_indices = list(tp_indices[0])

In [ ]:
# after identify true positive sample, create new loader for explaination

dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=1, num_workers=0)

In [ ]:
df = pd.read_csv(args.test_data_file)

In [ ]:
top_k_constant = [args.top_k_constant]

In [ ]:
def clean_special_token_values(all_values, padding=False):
    # special token in the beginning of the seq 
    all_values[0] = 0
    if padding:
        # get the last non-zero value which represents the att score for </s> token
        idx = [index for index, item in enumerate(all_values) if item != 0][-1]
        all_values[idx] = 0
    else:
        # special token in the end of the seq 
        all_values[-1] = 0
    return all_values
def get_word_att_scores(all_tokens: list, att_scores: list) -> list:
    word_att_scores = []
    for i in range(len(all_tokens)):
        token, att_score = all_tokens[i], att_scores[i]
        word_att_scores.append([token, att_score])
    return word_att_scores

In [ ]:
index = 0
progress_bar = tqdm(dataloader, total=len(dataloader))
with torch.no_grad():
    for mini_batch in progress_bar:
        if index in tp_indices and index == 99:
            (input_ids, labels) = mini_batch
            ids = input_ids[0].detach().tolist()
            all_tokens = tokenizer.convert_ids_to_tokens(ids)
            all_tokens = [token.replace("Ġ", "") for token in all_tokens]
            all_tokens = [token.replace("ĉ", "Ċ") for token in all_tokens]
            
            prob, attentions = model(input_ids=input_ids, output_attentions=True)
            attentions = attentions[0][0]
            attention = None
            # go into the layer
            for i in range(len(attentions)):
                layer_attention = attentions[i]
                # summerize the values of each token dot other tokens
                layer_attention = sum(layer_attention)
                if attention is None:
                    attention = layer_attention
                else:
                    attention += layer_attention
            # clean att score for <s> and </s>
            attention = clean_special_token_values(attention, padding=True)
            # attention should be 1D tensor with seq length representing each token's attention value
            # word_att_scores -> [[token, att_value], [token, att_value], ...]
            word_att_scores = get_word_att_scores(all_tokens=all_tokens, att_scores=attention)


            # go through each line
            separator = ["Ċ", " Ċ", "ĊĊ", " ĊĊ"]
            score_sum = 0
            line = ""
            score_sum = 0
            lines_with_score = []
            line_idx = 0
            for i in range(len(word_att_scores)):
                score_sum += word_att_scores[i][1]
                if word_att_scores[i][0] not in separator:
                    line += word_att_scores[i][0]
                else:
                    lines_with_score.append((line_idx, line, score_sum.detach().item()))
                    line = ""
                    score_sum = 0
                    line_idx += 1
            break
        index += 1

In [ ]:
line_idx

In [ ]:
sorted_lines = sorted(lines_with_score, key=lambda x: x[2], reverse=True)
sorted_lines[:int(0.15*line_idx)]

In [ ]:
import codecs

context = df.iloc[99]['func_before']
modified_context = codecs.decode(context, 'unicode_escape')

new_variable = modified_context.replace(r'\n', '\n')

print(new_variable)


In [ ]:
len(lines_with_score)

In [ ]:
df.iloc[99]